#### Copyright 2019 Die TensorFlow-Autoren.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Klassifizierung nach unausgeglichenen Daten

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/imbalanced_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ansicht auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Führen Sie in Google Colab aus</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notizbuch herunterladen</a></td>
</table>

Dieses Tutorial zeigt, wie ein stark unausgeglichenes Dataset klassifiziert wird, bei dem die Anzahl der Beispiele in einer Klasse die Anzahl der Beispiele in einer anderen Klasse erheblich übertrifft. Sie arbeiten mit dem auf Kaggle gehosteten Datensatz zur [Erkennung](https://www.kaggle.com/mlg-ulb/creditcardfraud) von Kreditkartenbetrug. Ziel ist es, lediglich 492 betrügerische Transaktionen von insgesamt 284.807 Transaktionen aufzudecken. Sie verwenden [Keras](../../guide/keras/overview.ipynb) , um das Modell und die [Klassengewichte](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model) zu definieren, damit das Modell aus den unausgeglichenen Daten lernen kann. .

Dieses Tutorial enthält vollständigen Code für:

- Laden Sie eine CSV-Datei mit Pandas.
- Erstellen Sie Zug-, Validierungs- und Testsätze.
- Definieren und trainieren Sie ein Modell mit Keras (einschließlich des Festlegens von Klassengewichten).
- Bewerten Sie das Modell anhand verschiedener Metriken (einschließlich Präzision und Rückruf).
- Probieren Sie gängige Techniken für den Umgang mit unausgeglichenen Daten aus:
    - Klassengewichtung
    - Überabtastung


## Konfiguration

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Datenverarbeitung und Exploration

### Laden Sie den Datensatz "Kaggle Credit Card Fraud" herunter

Pandas ist eine Python-Bibliothek mit vielen hilfreichen Dienstprogrammen zum Laden und Arbeiten mit strukturierten Daten und kann zum Herunterladen von CSVs in einen Datenrahmen verwendet werden.

Hinweis: Dieser Datensatz wurde im Rahmen einer Forschungskooperation von Worldline und der [Machine Learning Group](http://mlg.ulb.ac.be) der ULB (Université Libre de Bruxelles) zum Thema Big Data Mining und Betrugserkennung gesammelt und analysiert. Weitere Details zu aktuellen und früheren Projekten zu verwandten Themen finden Sie [hier](https://www.researchgate.net/project/Fraud-detection-5) und auf der Seite des [DefeatFraud-](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) Projekts

In [ ]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

In [ ]:
raw_df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V26', 'V27', 'V28', 'Amount', 'Class']].describe()

### Untersuchen Sie das Ungleichgewicht der Klassenbezeichnungen

Schauen wir uns das Ungleichgewicht der Datensätze an:

In [ ]:
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Dies zeigt den kleinen Anteil positiver Proben.

### Bereinigen, teilen und normalisieren Sie die Daten

Die Rohdaten haben einige Probleme. Erstens sind die Spalten `Time` und `Amount` zu variabel, um direkt verwendet zu werden. Löschen Sie die Spalte `Time` (da nicht klar ist, was dies bedeutet) und nehmen Sie das Protokoll der Spalte `Amount` , um den Bereich zu verringern.

In [ ]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps=0.001 # 0 => 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

Teilen Sie den Datensatz in Zug-, Validierungs- und Testsätze auf. Der Validierungssatz wird während der Modellanpassung verwendet, um den Verlust und alle Metriken zu bewerten. Das Modell passt jedoch nicht zu diesen Daten. Der Testsatz wird während der Trainingsphase nicht verwendet und erst am Ende verwendet, um zu bewerten, wie gut sich das Modell auf neue Daten verallgemeinert. Dies ist besonders wichtig bei unausgeglichenen Datensätzen, bei denen eine [Überanpassung](https://developers.google.com/machine-learning/crash-course/generalization/peril-of-overfitting) aufgrund des Mangels an Trainingsdaten ein erhebliches Problem darstellt.

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

Normalisieren Sie die Eingabefunktionen mit dem sklearn StandardScaler. Dadurch wird der Mittelwert auf 0 und die Standardabweichung auf 1 gesetzt.

Hinweis: Der `StandardScaler` nur mit `train_features` , um sicherzustellen, dass das Modell nicht auf die Validierungs- oder Testsätze blickt. 

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)


Achtung: Wenn Sie ein Modell bereitstellen möchten, ist es wichtig, dass Sie die Vorverarbeitungsberechnungen beibehalten. Der einfachste Weg, sie als Ebenen zu implementieren und sie vor dem Export an Ihr Modell anzuhängen.


### Schauen Sie sich die Datenverteilung an

Vergleichen Sie als nächstes die Verteilungen der positiven und negativen Beispiele über einige Merkmale. Gute Fragen, die Sie sich an dieser Stelle stellen sollten, sind:

- Sind diese Verteilungen sinnvoll?
    - Ja. Sie haben die Eingabe normalisiert und diese konzentrieren sich hauptsächlich auf den Bereich `+/- 2` .
- Können Sie den Unterschied zwischen den Verteilungen erkennen?
    - Ja, die positiven Beispiele enthalten eine viel höhere Rate an Extremwerten.

In [ ]:
pos_df = pd.DataFrame(train_features[ bool_train_labels], columns = train_df.columns)
neg_df = pd.DataFrame(train_features[~bool_train_labels], columns = train_df.columns)

sns.jointplot(pos_df['V5'], pos_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
plt.suptitle("Positive distribution")

sns.jointplot(neg_df['V5'], neg_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
_ = plt.suptitle("Negative distribution")

## Definieren Sie das Modell und die Metriken

Definieren Sie eine Funktion, die ein einfaches neuronales Netzwerk mit einer dicht verbundenen verborgenen Schicht, einer [Dropout-](https://developers.google.com/machine-learning/glossary/#dropout_regularization) Schicht zur Reduzierung von Überanpassungen und einer Ausgabe-Sigmoid-Schicht erstellt, die die Wahrscheinlichkeit einer betrügerischen Transaktion zurückgibt: 

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

def make_model(metrics = METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

### Nützliche Metriken verstehen

Beachten Sie, dass oben einige Metriken definiert sind, die vom Modell berechnet werden können und bei der Bewertung der Leistung hilfreich sind.

- **Falsch** negative und **falsch** positive Ergebnisse sind Proben, die **falsch** klassifiziert wurden
- **Echte** Negative und **echte** Positive sind Proben, die **korrekt** klassifiziert wurden
- **Die Genauigkeit** ist der Prozentsatz der korrekt klassifizierten Beispiele

> $ \ frac {\ text {true samples}} {\ text {total samples}} $

- **Präzision** ist der Prozentsatz der **vorhergesagten** Positiven, die korrekt klassifiziert wurden

> $ \ frac {\ text {true positive}} {\ text {true positive + false positive}} $

- **Rückruf** ist der Prozentsatz der **tatsächlich** positiven Ergebnisse, die korrekt klassifiziert wurden

> $ \ frac {\ text {wahre Positive}} {\ text {wahre Positive + falsche Negative}} $

- **AUC** bezieht sich auf die Fläche unter der Kurve einer Empfänger-Betriebskennlinie (ROC-AUC). Diese Metrik entspricht der Wahrscheinlichkeit, dass ein Klassifikator eine zufällige positive Stichprobe höher einstuft als eine zufällige negative Stichprobe.

Hinweis: Die Genauigkeit ist keine hilfreiche Metrik für diese Aufgabe. Sie können bei dieser Aufgabe eine Genauigkeit von 99,8% + erreichen, indem Sie ständig Falsch vorhersagen.

Weiterlesen:

- [Richtig gegen Falsch und Positiv gegen Negativ](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)
- [Richtigkeit](https://developers.google.com/machine-learning/crash-course/classification/accuracy)
- [Präzision und Rückruf](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)
- [ROC-AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

## Basismodell

### Erstellen Sie das Modell

Erstellen und trainieren Sie nun Ihr Modell mit der zuvor definierten Funktion. Beachten Sie, dass das Modell mit einer Chargengröße von 2048 angepasst wird. Dies ist wichtig, um sicherzustellen, dass jede Charge eine gute Chance hat, einige positive Proben zu enthalten. Wenn die Stapelgröße zu klein wäre, hätten sie wahrscheinlich keine betrügerischen Transaktionen, aus denen sie lernen könnten.

Hinweis: Dieses Modell wird das Klassenungleichgewicht nicht gut bewältigen. Sie werden es später in diesem Tutorial verbessern.

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
model = make_model()
model.summary()

Testen Sie das Modell:

In [ ]:
model.predict(train_features[:10])

### Optional: Stellen Sie die richtige Anfangsvorspannung ein.

Diese anfänglichen Vermutungen sind nicht großartig. Sie wissen, dass der Datensatz nicht ausgeglichen ist. Stellen Sie die Vorspannung der Ausgabeebene so ein, dass sie dies widerspiegelt (siehe: [Ein Rezept für das Training neuronaler Netze: "init well"](http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines) ). Dies kann bei der anfänglichen Konvergenz hilfreich sein.

Bei der Standard-Bias-Initialisierung sollte der Verlust bei `math.log(2) = 0.69314` 

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Die richtige Einstellung zum Einstellen kann abgeleitet werden aus:

$$ p_0 = pos / (pos + neg) = 1 / (1 + e ^ {- b_0}) $$ $$ b_0 = -log_e (1 / p_0 - 1) $$ $$ b_0 = log_e (pos / neg ) $$

In [ ]:
initial_bias = np.log([pos/neg])
initial_bias

Stellen Sie dies als anfängliche Verzerrung ein, und das Modell gibt viel vernünftigere anfängliche Vermutungen.

Es sollte in der Nähe sein: `pos/total = 0.0018`

In [ ]:
model = make_model(output_bias = initial_bias)
model.predict(train_features[:10])

Bei dieser Initialisierung sollte der anfängliche Verlust ungefähr:

$$ - p_0log (p_0) - (1-p_0) log (1-p_0) = 0,01317 $$

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Dieser anfängliche Verlust ist ungefähr 50-mal geringer als bei einer naiven Initialisierung.

Auf diese Weise muss das Modell nicht die ersten paar Epochen damit verbringen, zu lernen, dass positive Beispiele unwahrscheinlich sind. Dies erleichtert auch das Lesen von Plots des Verlusts während des Trainings.

### Überprüfen Sie die Anfangsgewichte

Um die verschiedenen Trainingsläufe vergleichbarer zu machen, sollten Sie die Gewichte dieses Ausgangsmodells in einer Prüfpunktdatei aufbewahren und vor dem Training in jedes Modell laden.

In [ ]:
initial_weights = os.path.join(tempfile.mkdtemp(),'initial_weights')
model.save_weights(initial_weights)

### Vergewissern Sie sich, dass der Bias Fix hilft

Vergewissern Sie sich vor dem Fortfahren schnell, dass die sorgfältige Bias-Initialisierung tatsächlich geholfen hat.

Trainieren Sie das Modell für 20 Epochen mit und ohne diese sorgfältige Initialisierung und vergleichen Sie die Verluste: 

In [ ]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
model = make_model()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
def plot_loss(history, label, n):
  # Use a log scale to show the wide range of values.
  plt.semilogy(history.epoch,  history.history['loss'],
               color=colors[n], label='Train '+label)
  plt.semilogy(history.epoch,  history.history['val_loss'],
          color=colors[n], label='Val '+label,
          linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  
  plt.legend()

In [ ]:
plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

Die obige Abbildung macht deutlich: In Bezug auf den Validierungsverlust bietet diese sorgfältige Initialisierung bei diesem Problem einen klaren Vorteil. 

### Trainiere das Modell

In [ ]:
model = make_model()
model.load_weights(initial_weights)
baseline_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels))

### Überprüfen Sie die Trainingshistorie

In diesem Abschnitt erstellen Sie Diagramme der Genauigkeit und des Verlusts Ihres Modells im Trainings- und Validierungssatz. Diese sind nützlich, um nach Überanpassungen zu suchen. Weitere Informationen hierzu finden Sie in diesem [Lernprogramm](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) .

Darüber hinaus können Sie diese Diagramme für jede der oben erstellten Metriken erstellen. Als Beispiel sind falsche Negative enthalten.

In [ ]:
def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()


In [ ]:
plot_metrics(baseline_history)

Hinweis: Die Validierungskurve ist im Allgemeinen besser als die Trainingskurve. Dies wird hauptsächlich durch die Tatsache verursacht, dass die Dropout-Schicht bei der Bewertung des Modells nicht aktiv ist.

### Metriken auswerten

Sie können eine [Verwirrungsmatrix verwenden,](https://developers.google.com/machine-learning/glossary/#confusion_matrix) um die tatsächlichen und vorhergesagten Beschriftungen zusammenzufassen, wobei die X-Achse die vorhergesagte Beschriftung und die Y-Achse die tatsächliche Beschriftung ist.

In [ ]:
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

Bewerten Sie Ihr Modell im Testdatensatz und zeigen Sie die Ergebnisse für die oben erstellten Metriken an.

In [ ]:
baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_baseline)

Wenn das Modell alles perfekt vorhergesagt hätte, wäre dies eine [Diagonalmatrix,](https://en.wikipedia.org/wiki/Diagonal_matrix) bei der Werte außerhalb der Hauptdiagonale, die auf falsche Vorhersagen hinweisen, Null wären. In diesem Fall zeigt die Matrix, dass Sie relativ wenige Fehlalarme haben, was bedeutet, dass es relativ wenige legitime Transaktionen gab, die falsch gekennzeichnet wurden. Trotz der Kosten für die Erhöhung der Anzahl falsch positiver Ergebnisse möchten Sie wahrscheinlich noch weniger falsche Negative haben. Dieser Kompromiss kann vorzuziehen sein, da durch falsche Negative betrügerische Transaktionen durchgeführt werden können, während durch falsche Positive eine E-Mail an einen Kunden gesendet werden kann, in der er aufgefordert wird, seine Kartenaktivität zu überprüfen.

### Zeichnen Sie den ROC

Zeichnen Sie nun den [ROC](https://developers.google.com/machine-learning/glossary#ROC) . Dieses Diagramm ist nützlich, da es auf einen Blick den Leistungsbereich zeigt, den das Modell durch einfaches Einstellen des Ausgabeschwellenwerts erreichen kann.

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

Es sieht so aus, als ob die Genauigkeit relativ hoch ist, aber der Rückruf und die Fläche unter der ROC-Kurve (AUC) sind nicht so hoch, wie Sie vielleicht möchten. Klassifizierer stehen häufig vor Herausforderungen, wenn sie versuchen, sowohl die Präzision als auch den Rückruf zu maximieren. Dies gilt insbesondere für die Arbeit mit unausgeglichenen Datensätzen. Es ist wichtig, die Kosten für verschiedene Arten von Fehlern im Zusammenhang mit dem Problem zu berücksichtigen, das Sie interessiert. In diesem Beispiel kann ein falsches Negativ (eine betrügerische Transaktion wird übersehen) finanzielle Kosten verursachen, während ein falsches Positiv (eine Transaktion wird fälschlicherweise als betrügerisch gekennzeichnet) die Benutzerzufriedenheit verringern kann.

## Klassengewichte

### Klassengewichte berechnen

Das Ziel besteht darin, betrügerische Transaktionen zu identifizieren, aber Sie haben nicht sehr viele dieser positiven Stichproben, mit denen Sie arbeiten können. Daher möchten Sie, dass der Klassifizierer die wenigen verfügbaren Beispiele stark gewichtet. Sie können dies tun, indem Sie Keras-Gewichte für jede Klasse über einen Parameter übergeben. Dies führt dazu, dass das Modell Beispielen aus einer unterrepräsentierten Klasse "mehr Aufmerksamkeit" schenkt.

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

### Trainiere ein Modell mit Klassengewichten

Versuchen Sie nun, das Modell mit Klassengewichten neu zu trainieren und zu bewerten, um festzustellen, wie sich dies auf die Vorhersagen auswirkt.

Note: Using `class_weights` changes the range of the loss. This may affect the stability of the training depending on the optimizer. Optimizers whose step size is dependent on the magnitude of the gradient, like `optimizers.SGD`, may fail. The optimizer used here, `optimizers.Adam`, is unaffected by the scaling change. Also note that because of the weighting, the total losses are not comparable between the two models.

In [ ]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels),
    # The class weights go here
    class_weight=class_weight) 

### Überprüfen Sie die Trainingshistorie

In [ ]:
plot_metrics(weighted_history)

### Metriken auswerten

In [ ]:
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
weighted_results = weighted_model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

Hier können Sie sehen, dass bei Klassengewichten die Genauigkeit und Präzision geringer sind, weil es mehr falsch positive Ergebnisse gibt, aber umgekehrt sind der Rückruf und die AUC höher, weil das Modell auch mehr wahr positive Ergebnisse gefunden hat. Trotz geringerer Genauigkeit weist dieses Modell einen höheren Rückruf auf (und identifiziert betrügerischere Transaktionen). Natürlich sind beide Fehlertypen mit Kosten verbunden (Sie möchten Benutzer auch nicht nerven, indem Sie zu viele legitime Transaktionen als betrügerisch kennzeichnen). Berücksichtigen Sie sorgfältig die Kompromisse zwischen diesen verschiedenen Arten von Fehlern für Ihre Anwendung.

### Zeichnen Sie den ROC

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right')

## Überabtastung

### Überprobe die Minderheitsklasse

Ein verwandter Ansatz wäre das erneute Abtasten des Datensatzes durch Überabtasten der Minderheitsklasse.

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

#### Verwenden von NumPy

Sie können den Datensatz manuell ausgleichen, indem Sie aus den positiven Beispielen die richtige Anzahl von Zufallsindizes auswählen:

In [ ]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

#### Verwenden von `tf.data`

If you're using `tf.data` the easiest way to produce balanced examples is to start with a `positive` and a `negative` dataset, and merge them. See [the tf.data guide](../../guide/data.ipynb) for more examples.

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

Jeder Datensatz enthält `(feature, label)` Paare:

In [ ]:
for features, label in pos_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

Führen Sie die beiden mithilfe von `experimental.sample_from_datasets` :

In [ ]:
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

Um diesen Datensatz verwenden zu können, benötigen Sie die Anzahl der Schritte pro Epoche.

Die Definition von "Epoche" ist in diesem Fall weniger klar. Angenommen, es ist die Anzahl der Stapel, die erforderlich sind, um jedes negative Beispiel einmal zu sehen:

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

### Trainieren Sie die überabgetasteten Daten

Versuchen Sie nun, das Modell mit dem neu abgetasteten Datensatz zu trainieren, anstatt Klassengewichte zu verwenden, um zu sehen, wie diese Methoden verglichen werden.

Hinweis: Da die Daten durch Replizieren der positiven Beispiele ausgeglichen wurden, ist die Gesamtgröße des Datensatzes größer und jede Epoche wird für weitere Trainingsschritte ausgeführt. 

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks = [early_stopping],
    validation_data=val_ds)

Wenn der Trainingsprozess bei jeder Gradientenaktualisierung den gesamten Datensatz berücksichtigen würde, wäre diese Überabtastung im Wesentlichen identisch mit der Klassengewichtung.

Wenn Sie das Modell jedoch chargenweise trainieren, wie Sie es hier getan haben, liefern die überabgetasteten Daten ein weicheres Gradientensignal: Anstatt jedes positive Beispiel in einer Charge mit großem Gewicht zu zeigen, werden sie jedes Mal in vielen verschiedenen Chargen mit einem angezeigt kleines Gewicht.

Dieses weichere Gradientensignal erleichtert das Trainieren des Modells.

### Überprüfen Sie die Trainingshistorie

Beachten Sie, dass die Verteilung der Metriken hier unterschiedlich ist, da die Trainingsdaten eine völlig andere Verteilung aufweisen als die Validierungs- und Testdaten. 

In [ ]:
plot_metrics(resampled_history )

### Umschulen


Da das Training mit den ausgeglichenen Daten einfacher ist, kann das oben beschriebene Trainingsverfahren schnell überanpassen.

So break up the epochs to give the `callbacks.EarlyStopping` finer control over when to stop training.

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

resampled_history = resampled_model.fit(
    resampled_ds,
    # These are not real epochs
    steps_per_epoch = 20,
    epochs=10*EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_ds))

### Überprüfen Sie die Trainingshistorie erneut

In [ ]:
plot_metrics(resampled_history)

### Metriken auswerten

In [ ]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_resampled)

### Zeichnen Sie den ROC

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_roc("Train Resampled", train_labels, train_predictions_resampled,  color=colors[2])
plot_roc("Test Resampled", test_labels, test_predictions_resampled,  color=colors[2], linestyle='--')
plt.legend(loc='lower right')

## Anwenden dieses Tutorials auf Ihr Problem

Eine unausgewogene Datenklassifizierung ist von Natur aus eine schwierige Aufgabe, da es so wenige Beispiele gibt, aus denen man lernen kann. Sie sollten immer zuerst mit den Daten beginnen und Ihr Bestes tun, um so viele Stichproben wie möglich zu sammeln und gründlich darüber nachzudenken, welche Funktionen relevant sein können, damit das Modell Ihre Minderheitenklasse optimal nutzen kann. Irgendwann kann es sein, dass Ihr Modell Schwierigkeiten hat, die gewünschten Ergebnisse zu verbessern und zu erzielen. Daher ist es wichtig, den Kontext Ihres Problems und die Kompromisse zwischen verschiedenen Arten von Fehlern zu berücksichtigen.